In [1]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.pivot_table(df, values='a', index='b', columns='c', aggfunc='sum', fill_value=0)
# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [2]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [3]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'hui_for_web.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [4]:
# hui= r'.\inputs\housing_unit_inventory_2022.gdb\housing_unit_inventory_2022'
hui= r".\inputs\housing_unit_inventory_2022_20240122.gdb\housing_unit_inventory_2022"
t = r'.\inputs\Stations_Interchanges.gdb\Interchanges_and_Stations'
t_lyr = arcpy.MakeFeatureLayer_management(t, 't_lyr')
parks = r".\inputs\wcv_parks.shp"
trails = r".\inputs\TrailsAndPathways_WFRCMAG.shp"
trails_lyr = arcpy.MakeFeatureLayer_management(trails, 'trails_lyr')
centers = r"E:\Data\Boundaries\WC2050Centers.shp"

In [5]:
# use spatial join to summarize h+t
target_features = hui
join_features = centers
output_features = os.path.join(gdb, "_00_hui_center_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

fields = ['AreaName']
for f in fields:

# field
    fieldindex = fieldmappings.findFieldMapIndex(f)
    fieldmap = fieldmappings.getFieldMap(fieldindex)
    fieldmap.mergeRule = 'first'
    fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_COMMON", 
                           fieldmappings, "HAVE_THEIR_CENTER_IN")

In [6]:
sj_df = pd.DataFrame.spatial.from_featureclass(sj[0])
# sj_df.head()

In [7]:
sj_df = sj_df[['UNIT_ID', 'AreaName', 'AreaType']].copy()
sj_df.columns = ['UNIT_ID', 'CENTER', 'CENTERTYPE']

In [8]:
name = 'parks'
# copy = arcpy.conversion.FeatureClassToFeatureClass(hui, gdb, f'hui_near_{name}')
# near_result = arcpy.analysis.Near(in_features=copy, near_features=parks, method='GEODESIC')
# df_parks = pd.DataFrame.spatial.from_featureclass(near_result[0])
df_parks = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, f'hui_near_{name}'))
# df_parks.head()

In [9]:
name = 'trails'
# copy = arcpy.conversion.FeatureClassToFeatureClass(hui, gdb, f'hui_near_{name}')
# arcpy.SelectLayerByAttribute_management(trails_lyr, 'NEW_SELECTION', """Status IN ('EXISTING', 'Existing', 'Current')""")
# near_result = arcpy.analysis.Near(in_features=copy, near_features=trails_lyr, method='GEODESIC')
# df_trails = pd.DataFrame.spatial.from_featureclass(near_result[0])
df_trails = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, f'hui_near_{name}'))
# df_trails.head()

In [10]:
name = 'frontrunner'
# copy = arcpy.conversion.FeatureClassToFeatureClass(hui, gdb, f'hui_near_{name}')
# arcpy.SelectLayerByAttribute_management(t_lyr, 'NEW_SELECTION', "SubMode = 'Commuter Rail Station' AND Status = 'Current'")
# near_result = arcpy.analysis.Near(in_features=copy, near_features=t_lyr, method='GEODESIC')
# df_fr = pd.DataFrame.spatial.from_featureclass(near_result[0])
df_fr = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, f'hui_near_{name}'))
# df_fr.head()

In [11]:
name = 'lightrail'
# copy = arcpy.conversion.FeatureClassToFeatureClass(hui, gdb, f'hui_near_{name}')
# arcpy.SelectLayerByAttribute_management(t_lyr, 'NEW_SELECTION', "SubMode = 'Light Rail Station' AND Status = 'Current'")
# near_result = arcpy.analysis.Near(in_features=copy, near_features=t_lyr, method='GEODESIC')
# df_lr = pd.DataFrame.spatial.from_featureclass(near_result[0])
df_lr = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, f'hui_near_{name}'))
# df_lr.head()

In [12]:
name = 'brt'
# copy = arcpy.conversion.FeatureClassToFeatureClass(hui, gdb, f'hui_near_{name}')
# arcpy.SelectLayerByAttribute_management(t_lyr, 'NEW_SELECTION', "SubMode = 'BRT Stop' And (Status = 'Current' Or Status IS NULL)")
# near_result = arcpy.analysis.Near(in_features=copy, near_features=t_lyr, method='GEODESIC')
# df_brt = pd.DataFrame.spatial.from_featureclass(near_result[0])
df_brt = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, f'hui_near_{name}'))
# df_brt.head()

In [13]:
name = 'fwyexit'
# copy = arcpy.conversion.FeatureClassToFeatureClass(hui, gdb, f'hui_near_{name}')
# arcpy.SelectLayerByAttribute_management(t_lyr, 'NEW_SELECTION', "SubMode = 'Interchange' AND Status = 'Current'")
# near_result = arcpy.analysis.Near(in_features=copy, near_features=t_lyr, method='GEODESIC')
# df_fwy = pd.DataFrame.spatial.from_featureclass(near_result[0])
df_fwy = pd.DataFrame.spatial.from_featureclass(os.path.join(gdb, f'hui_near_{name}'))
# df_fwy.head()

In [14]:
# convert distance from meters to miles
df_fr['DIST_FR'] = df_fr['NEAR_DIST']*.000621371
df_lr['DIST_LR'] = df_lr['NEAR_DIST']*.000621371
df_brt['DIST_BRT'] = df_brt['NEAR_DIST']*.000621371
df_fwy['DIST_FWYE'] = df_fwy['NEAR_DIST']*.000621371
df_parks['DIST_PARK'] = df_parks['NEAR_DIST']*.000621371
df_trails['DIST_TRAIL'] = df_trails['NEAR_DIST']*.000621371

df_fr['DIST_FR'] = round(df_fr['DIST_FR'], 2)
df_lr['DIST_LR'] = round(df_lr['DIST_LR'], 2)
df_brt['DIST_BRT'] = round(df_brt['DIST_BRT'], 2)
df_fwy['DIST_FWYE'] = round(df_fwy['DIST_FWYE'], 2)
df_parks['DIST_PARK'] = round(df_parks['DIST_PARK'], 2)
df_trails['DIST_TRAIL'] = round(df_trails['DIST_TRAIL'], 2)

df_fr = df_fr[['UNIT_ID', 'DIST_FR']].copy()
df_lr = df_lr[['UNIT_ID', 'DIST_LR']].copy()
df_brt = df_brt[['UNIT_ID', 'DIST_BRT']].copy()
df_fwy = df_fwy [['UNIT_ID', 'DIST_FWYE']].copy()
df_parks = df_parks[['UNIT_ID', 'DIST_PARK']].copy()
df_trails = df_trails[['UNIT_ID', 'DIST_TRAIL']].copy()

In [16]:
hui_df = pd.DataFrame.spatial.from_featureclass(hui)
hui_df = (hui_df.merge(sj_df, on='UNIT_ID', how='left') 
                .merge(df_fr, on='UNIT_ID', how='left') 
                .merge(df_lr, on='UNIT_ID', how='left')  
                .merge(df_brt, on='UNIT_ID', how='left')
                .merge(df_fwy, on='UNIT_ID', how='left')
                .merge(df_parks, on='UNIT_ID', how='left')
                .merge(df_trails, on='UNIT_ID', how='left')) 

# hui_sf = hui_df[hui_df['TYPE']== 'single_family'].copy()
# hui_mf = hui_df[hui_df['TYPE']!= 'single_family'].copy()

# hui_df2 = pd.concat([hui_sf, hui_mf])
hui_df.spatial.to_featureclass(location=os.path.join(gdb2, 'hui_2022_web_version'),sanitize_columns=False)

'e:\\Projects\\Housing-Unit-Inventory-Explorer\\python\\Outputs\\hui_for_web.gdb\\hui_2022_web_version'

In [17]:
arcpy.management.FeatureToPoint(os.path.join(gdb2, 'hui_2022_web_version'), os.path.join(gdb2, 'hui_2022_pts_web_version'), "INSIDE")

<Result '.\\Outputs\\hui_for_web.gdb\\hui_2022_pts_web_version'>

# exploration

In [18]:
# hui_df = pd.DataFrame.spatial.from_featureclass(hui)

In [19]:
# hui_df.hist(column='DUA')